# CS109b Final Project

## Milestone2

by Danqing Wang, Wenshan Zheng, Zecai Liang

----------

In [1]:
# import libraries
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import urllib
from bs4 import BeautifulSoup
import time
import re
import tmdbsimple as tmdb
tmdb.API_KEY = '4074d0170761c40d9c07d9016ddd4965'
from collections import Counter

from imdb import IMDb
ia = IMDb()

import sys


import warnings
warnings.filterwarnings('ignore')

---
## Part1. Download Data

For demenstration of code: we use 100 movies

### 1.1 Download Data from TMDB

 #### 1.1.1 Get list of IMDB and TMDB IDs 

In [5]:
# set up some basic url link strings, to be used later
APIKeyZ = "api_key=4074d0170761c40d9c07d9016ddd4965"
base_url_search = "https://api.themoviedb.org/3/discover/movie?"
popular_desc = "&sort_by=popularity.desc"
year =  "&primary_release_year={}"
page_number = '&page={}'
query_url = 'https://api.themoviedb.org/3/movie/{}?'

In [6]:
# query latest movie and get the largest ID from TMDB
latest_url =  query_url.format("latest") + APIKeyZ
page = urllib.urlopen(latest_url).read()
soup = BeautifulSoup(page, "lxml")
prettified = soup.prettify()
movie_index = prettified.find(',"id"')
j_beginning = 6
movie_id_latest = ''
while (prettified[movie_index + j_beginning].isdigit()):
    movie_id_latest += str(prettified[movie_index + j_beginning])
    j_beginning += 1
movie_id_latest = int(movie_id_latest)

In [18]:
# randomly generate 200K movies ids without replacement to get movie TMDB IDs (it's possible that some of these IDs return empty)
movie_sample = 200000
movie_list = np.random.choice(range(1, movie_id_latest), movie_sample, replace = False)

In [19]:
# get movies' IMDB and TMDB ID, to be used for further search of IMDB ID
time.sleep(10)
IMDB_id_list = []
TMDB_id_list = []

for i in range(len(movie_list)):
    movie_url =  query_url.format(movie_list[i]) + APIKeyZ
    page = urllib.urlopen(movie_url).read()
    soup = BeautifulSoup(page, "lxml")
    prettified = soup.prettify()
    movie_index = prettified.find('"imdb_id"')
    if movie_index != -1:      
        j_beginning = 13
        movie_id_temp = ''
        while (prettified[movie_index + j_beginning].isdigit()):
            movie_id_temp += str(prettified[prettified.find('"imdb_id"') + j_beginning])
            j_beginning += 1
        if len(movie_id_temp) == 7:
            IMDB_id_list += [movie_id_temp]
            TMDB_id_list += [movie_list[i]]
            
    if len(IMDB_id_list) >= 100: # we use 25K for real data construction
        break    
    if i % 40 == 39:
        time.sleep(11)

In [21]:
# Write out movie TMDB IDs
thefile = open('tmdb_id_demo.txt', 'w')
for item in TMDB_id_list:
    thefile.write("%s\n" % item)
thefile.close()

In [22]:
# Write out movie IMDB IDs
thefile = open('imdb_id_demo.txt', 'w')
for item in IMDB_id_list:
    thefile.write("%s\n" % item)
thefile.close()

#### 1.1.2 Read in TMDB ID and get all TMDB information

In [23]:
text_file = open('tmdb_id_demo.txt', 'r')
tmdb_id_list = text_file.read().split('\n')
text_file.close()

In [24]:
# search one movie by ID and create list of column names
movie = tmdb.Movies(tmdb_id_list[0])
response = movie.info()
columns = []
for i in range(len(response)):
    column_temp = [str(response.items()[i][0])]
    columns += column_temp

In [25]:
# create a dataframe to store information of top 500 popular movies
index = range(len(tmdb_id_list)) 
df1 = pd.DataFrame(index = index, columns=columns)
df1 = df1.fillna(0)

In [35]:
# store information into dataframe, some information received is stroed as a list thus we convert it into strings
time.sleep(10)
for i in range(len(index) - 1):    
    movie = tmdb.Movies(tmdb_id_list[i])
    response = movie.info()
    if i % 40 == 39:
        time.sleep(11)
    for j in range(0, len(columns)):
        if j == 1:
            country_temp = []
            for k in range(len(response[columns[j]])):
                country_temp += [(response[columns[j]][k]['name'])]
            df1.iloc[i,j] = str(country_temp)
        elif j == 6:
            genre_temp = []
            for k in range(len(response[columns[j]])):
                genre_temp += [(response[columns[j]][k]['name'])]
            df1.iloc[i,j] = str(genre_temp)
        elif j == 11:
            if response[columns[j]] != None:
                df1.iloc[i,j] = str(response[columns[j]])
        elif j == 14:
            language_temp = []
            for k in range(len(response[columns[j]])):
                language_temp += [(response[columns[j]][k]['iso_639_1'])]
            df1.iloc[i,j] = str(language_temp)
        elif j == 18:
            company_temp = []
            for k in range(len(response[columns[j]])):
                company_temp += [(response[columns[j]][k]['name'])]
            df1.iloc[i,j] = str(company_temp)
        else:
            df1.iloc[i,j] = response[columns[j]]
    if i % 1000 ==0:
        print i
df1.head()

0


,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/zPczhuSC1POwT12pEnfGm2HUj8O.jpg,[u'Argentina'],0,None,False,378574,[u'Comedy'],Disputas en la cama,None,0,...,tt0191090,False,None,[],1972-01-21,0.000005,Disputas en la cama,0,0.0,NaN
1,/tgY8b2ShG7IOIn5i3ykefoa4Rte.jpg,[u'Japan'],0,,False,317322,[],800 Two Lap Runners,,0,...,tt0109022,False,None,[],1994-07-09,0.000158,800 Two Lap Runners,0,0.0,110.0
2,None,[],0,"On November 25th 2005, Heaven 17 put on a rare...",False,100629,[],Heaven 17 - Live at La Scala,,0,...,tt1439310,False,None,[],,0.000000,Heaven 17 - Live at La Scala,0,0.0,90.0
3,/hvdR9bJgJsBqZjfaZwI8NdLqIH.jpg,[u'Philippines'],0,"Don Victorino Hernandez, a creole secular prie...",False,256951,"[u'Drama', u'Crime', u'History', u'Horror']",Angustia,,0,...,tt3326152,False,None,"[u'Cinema One Originals', u'Cine Rinconada']",2013-11-13,0.002508,Angustia,0,0.0,0.0
4,/aQBgs3f4jc5hJtgMHQcFPoug42s.jpg,[u'France'],0,"""Bartosch showed that animation could be poeti...",False,105773,[u'Animation'],The Idea,,6,...,tt0145834,False,/nRuC7ODVBapTQOanxbtFVLNcGws.jpg,[],1932-01-01,0.000608,L'idée,0,6.4,25.0


In [38]:
# write the dataframe out
df1.to_csv('TMDb_data_demo.csv', encoding='utf-8')

#### 1.1.3 Download posters

In [39]:
# read in TMDB dataframe
df1 = pd.read_csv('TMDb_data_demo.csv',index_col=0)
df1

,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/zPczhuSC1POwT12pEnfGm2HUj8O.jpg,[u'Argentina'],0,NaN,False,378574,[u'Comedy'],Disputas en la cama,NaN,0,...,tt0191090,False,NaN,[],1972-01-21,0.000005,Disputas en la cama,0,0.0,NaN
1,/tgY8b2ShG7IOIn5i3ykefoa4Rte.jpg,[u'Japan'],0,NaN,False,317322,[],800 Two Lap Runners,NaN,0,...,tt0109022,False,NaN,[],1994-07-09,0.000158,800 Two Lap Runners,0,0.0,110.0
2,NaN,[],0,"On November 25th 2005, Heaven 17 put on a rare...",False,100629,[],Heaven 17 - Live at La Scala,NaN,0,...,tt1439310,False,NaN,[],NaN,0.000000,Heaven 17 - Live at La Scala,0,0.0,90.0
3,/hvdR9bJgJsBqZjfaZwI8NdLqIH.jpg,[u'Philippines'],0,"Don Victorino Hernandez, a creole secular prie...",False,256951,"[u'Drama', u'Crime', u'History', u'Horror']",Angustia,NaN,0,...,tt3326152,False,NaN,"[u'Cinema One Originals', u'Cine Rinconada']",2013-11-13,0.002508,Angustia,0,0.0,0.0
4,/aQBgs3f4jc5hJtgMHQcFPoug42s.jpg,[u'France'],0,"""Bartosch showed that animation could be poeti...",False,105773,[u'Animation'],The Idea,NaN,6,...,tt0145834,False,/nRuC7ODVBapTQOanxbtFVLNcGws.jpg,[],1932-01-01,0.000608,L'idée,0,6.4,25.0
5,NaN,[u'Germany'],0,NaN,False,287104,[u'Drama'],Der Fall Maurizius,NaN,0,...,tt0162804,False,NaN,[u'Chamier Film'],1981-09-20,0.000266,Der Fall Maurizius,0,0.0,NaN
6,/5D2Lzw82q4YlvjrWL6vsbJMCmTD.jpg,[u'United States of America'],0,Terry faces off against a much younger Batman,False,269246,"[u'Action', u'Animation', u'Science Fiction']",Batman Beyond Darwyn Cooke's Batman 75th Anniv...,NaN,7,...,tt3643216,False,/ssUR1IOGVP4Mdu7lzcSiyCJNa5b.jpg,"[u'DC Comics', u'Warner Bros. Animation']",2014-04-19,0.120770,Batman Beyond Darwyn Cooke's Batman 75th Anniv...,0,8.3,1.0
7,NaN,[],0,Shows a large open barge loaded with people of...,False,190702,[],Emigrants Landing at Ellis Island,NaN,1,...,tt0419726,False,NaN,[],1903-08-14,0.000143,Emigrants Landing at Ellis Island,0,7.0,2.0
8,NaN,[u'Germany'],0,NaN,False,410551,[u'Drama'],Hütet eure Töchter!,NaN,0,...,tt0058219,False,NaN,[],1964-05-28,0.000151,Hütet eure Töchter!,0,0.0,NaN
9,/pwC2zotRlRuhABg3j8Z7TaYk4Hw.jpg,"[u'France', u'Italy']",0,Stuntman Johnny gets tired of putting his life...,False,3695,"[u'Action', u'Drama']",Stuntman,NaN,0,...,tt0063652,False,NaN,[u'Marianne Productions'],1968-01-01,0.072975,Stuntman,0,0.0,98.0


In [40]:
import urllib
path_poster = "http://image.tmdb.org/t/p/w500/{}"
path_poster_local = '{}.jpg'
poster_list = []

for i in range(len(df1)):
    extend_path = df1.loc[i, "poster_path"]
    if len(str(extend_path)) > 4:
        poster_list += [i]
        IMDB_ID = df1.loc[i, "imdb_id"]
        urllib.urlretrieve(path_poster.format(extend_path), path_poster_local.format(IMDB_ID))

### 1.2 Download Data from IMDB

In [4]:
def download_imdb_n(id_list = "imdb_id25K.txt", out_file = "data_imdb_top100.txt", n = 100):
    ##### function downloads top n entires of imdb data as given by imdb_id25K.txt
    ##### outputs downloaded data as 

    # Import TMDb top 25,000 movies using imdb_ids
    ID = np.array(pd.read_csv(id_list, header=None))
    ID = ID[:n]

    # Among the different variables, we select the following variables of interest in our analysis  
    columns = ['title','genres', 'director', 'distributors', 'year', 'rating', 'votes', 'runtimes', 
              'language codes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'kind', 
               'country codes', 'countries', 'cover url', 'aspect_ratio', 'production companies', 
               'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music',
               'canonical title', 'editorial department', 'canonical title', 'long imdb title',
               'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title',
               'full-size cover url']

    ## Download IMDb data
    index = range(1, len(ID)+1) 
    df = pd.DataFrame(index = index, columns=columns)
    df = df.fillna(0)

    # Fill in dataframe df 
    from imdb import IMDb
    ia = IMDb()
    for i in range(0, len(index)):
        movie = ia.get_movie(ID[i]) # grab movie data by id
        ia.update(movie)
        keys = movie.keys() # generate the available keys of this particular movie 
        print i

        for j in range(0, len(columns)):
            if columns[j] in keys:
                if type(movie[columns[j]]) == list:
                    result = str(movie[columns[j]])
                else:
                    result = movie[columns[j]]
                df.iloc[i,j] = result
            else:
                df.iloc[i,j] = 'nan'

    # add column if IDs to the dataframe
    df['imdb_ids'] = pd.Series(ID.reshape(n,), index = df.index)

    # export dataframe to txt file
    df.to_csv(out_file, encoding='utf-8')

In [ ]:
download_imdb_n(id_list = "imdb_id_demo.txt", out_file = "data_imdb_top100.txt", n = 100)

---
## Part2. Parse and Merge Data

For demenstration of code:
use the the top 100 rows

### 2.1 Parse IMDB Data

For the IMDB data (25K data points) we downloaded from IMDB server, 
we want to clean each column and extract the information.

- for response "genres", split multiple genres for one movie ans store them in a list
- for variables "runtimes", "language codes", "languages", "country codes", "countries": extract the multiple entries for one movie and store the result in a list
- for variables "director", "writer","cast","distributors", "producer", "production companies","cinematographer", "animation department", "original music", "editorial department": extract the multiple person IDs and company IDs for one movie and store the result in a list
- for variable "mpaa", we extract the MPAA rating for each movie, and store it under variable "mpaa"; we also keep the explaination of why the rating is made, and store the string under variable "mpaa_reason"

In [41]:
##### The function to parse data downloaded from IMDB
##### import the raw data by data_name, and specify the file name of the parsed data frame

def parse_imdb(data_name = "imdb_data.txt", new_name = "imdb_data_parse.txt"):

        import pandas as pd
        import numpy as np
        import re
        import warnings
        warnings.filterwarnings("ignore")

        #### load the imdb data to be parsed
        df = pd.read_csv(data_name)

        ## copy column "mpaa" to store the reason of MPAA rating
        df2 = df
        df2["mpaa_reason"] = df["mpaa"]

        
        #### select variables 
        #group0 = ["genres", "imdb_id"] 
        #group1 = ["title", "year", "rating", "votes", "top 250 rank", "kind"]
        #group2 = ["runtimes", "language codes", "languages", "country codes", "countries"]
        #group3 = ["mpaa", "mpaa_reason"]
        #group4 = ["director", "writer","cast","distributors", "producer", "production companies","cinematographer", "animation department", "original music", "editorial department"]
        
       
        
             
        #### for each row of the data
        for i in range(len(df)):

                ### -----------------------------------------------------------------------
                ## for variables with multiple items, return a list (later turn into dummy coding)

                for val in ["genres", "runtimes", "language codes", "languages", "country codes", "countries"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                                st = df.ix[i,val]
                                value = st[3:-2].split("', u'") 
                                df2[val][i] = value
  
                ### -----------------------------------------------------------------------
                ## extract movie MPAA rating as a string (later turn into factor)

                for val in ["mpaa"]:  
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = re.findall(r'Rated (.*?) for',st)[0] # extract the word between "Rated" and "for"
                            df2[val][i] = value
                ### -----------------------------------------------------------------------
                ## extract the reason of MPAA rating as a string (for text analysis later)

                for val in ["mpaa_reason"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = st.split("for ", 1)[1]
                            df2[val][i] = value
                ### -----------------------------------------------------------------------
                ## for variables in the dictionary format, return a list of:
                ##     - person id 
                ##     - company id (for "production companies" and "distributors")

                for val in ["director", "writer","cast", 
                            "distributors", "producer", "production companies", 
                            "cinematographer", "animation department", 
                                "original music", "editorial department"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = re.findall(r'\d+',st)
                            df2[val][i] = value
                            
                ### -----------------------------------------------------------------------
                ## variables that may not be useful:
                    # ["cover url", "full-size cover url"," canonical title", "canonical title.1", 
                    # "long imdb title", "long imdb canonical title", "smart canonical title", 
                    # "smart long imdb canonical title"]
                    
        
        df2.to_csv(new_name)


For demonstration purpose, here we use the top 100 rows of the database.

In [ ]:
parse_imdb(data_name = "imdb_top100_data.txt", new_name = "imdb_top100_data_parse.txt" )

In [42]:
imdb_top100_data_parse = pd.read_csv( "imdb_top100_data_parse.txt")
imdb_top100_data_parse.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,title,genres,director,distributors,year,rating,votes,...,canonical title,editorial department,canonical title.1,long imdb title,long imdb canonical title,smart canonical title,smart long imdb canonical title,full-size cover url,imdb_ids,mpaa_reason
0,0,0,0,Four Rooms,['Comedy'],"['0025978', '0734319', '0001675', '0000233']","['0051881', '0014703', '0080682', '0022594', '...",1995.0,6.7,81543.0,...,Four Rooms,"['0020317', '0003692', '0042901', '0120656', '...",Four Rooms,Four Rooms (1995),Four Rooms (1995),Four Rooms,Four Rooms (1995),https://images-na.ssl-images-amazon.com/images...,113101,"pervasive strong language, sexuality and some ..."
1,1,1,1,"Sonntag, im August",['Short'],['1262754'],NaN,2004.0,6.5,11.0,...,"Sonntag, im August",NaN,"Sonntag, im August","Sonntag, im August (2004)","Sonntag, im August (2004)","Sonntag, im August","Sonntag, im August (2004)",https://images-na.ssl-images-amazon.com/images...,425473,NaN
2,2,2,2,Star Wars,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",['0000184'],"['0000756', '0074275', '20', '0105985', '20', ...",1977.0,8.7,969951.0,...,Star Wars,"['0091250', '0300879', '0366165', '0385492', '...",Star Wars,Star Wars (1977),Star Wars (1977),Star Wars,Star Wars (1977),https://images-na.ssl-images-amazon.com/images...,76759,sci-fi violence and brief mild language
3,3,3,3,Finding Nemo,"['Animation', 'Adventure', 'Comedy', 'Family']","['0004056', '0881279']","['0358618', '0064622', '0000779', '0110125', '...",2003.0,8.1,743705.0,...,Finding Nemo,"['0023224', '1404655', '1733701', '1404694', '...",Finding Nemo,Finding Nemo (2003),Finding Nemo (2003),Finding Nemo,Finding Nemo (2003),https://images-na.ssl-images-amazon.com/images...,266543,NaN
4,4,4,4,The Dark,"['Horror', 'Mystery', 'Thriller']",['0269502'],"['0002257', '0059064', '0022594', '0049085', '...",2005.0,5.4,9415.0,...,"Dark, The","['1262315', '3024002', '1930271', '0996933', '...","Dark, The",The Dark (2005),"Dark, The (2005)","Dark, The","Dark, The (2005)",https://images-na.ssl-images-amazon.com/images...,411267,some violent/disturbing images and language


### 2.2 Merge TMDB and IMDB

#### 2.2.1 Merge features from TMDB and IMDB

Primarily, we get all our data from IMDB and TMDB. We combine all information by API requests from these 2 sources. After combining, we remove duplicated features such as title, language codes, runtimes, country codes. Further, we deleted features which are non-informative, such as kind, cover url, poster_path, id, backdrop_path. IMDB_ID is also non-informative for prediction but we keep it as a common link between poster names and our dataset. Some other duplicated features are also removed, such as language codes (it is the same as language), country codes (same as countries) and so on.

After these, remaining features for prediction are: 'title', 'director', 'distributors', 'year', 'rating', 'votes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'countries', 'aspect_ratio', 'production companies', 'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music', 'editorial department', 'imdb_ids', 'mpaa_reason', 'revenue_TMDB', 'overview_TMDB', 'tagline_TMDB', 'vote_count_TMDB', 'belongs_to_collection_TMDB', 'original_language_TMDB', 'status_TMDB',  'release_date_TMDB', 'popularity_TMDB', 'budget_TMDB', 'vote_average_TMDB' and 'runtime_TMDB'.
 

In [12]:
# import TMDB data
df_tmdb = pd.read_csv('TMDb_data_demo.csv',index_col=0)
df_tmdb

,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/zPczhuSC1POwT12pEnfGm2HUj8O.jpg,[u'Argentina'],0,NaN,False,378574,[u'Comedy'],Disputas en la cama,NaN,0,...,tt0191090,False,NaN,[],1972-01-21,0.000005,Disputas en la cama,0,0.0,NaN
1,/tgY8b2ShG7IOIn5i3ykefoa4Rte.jpg,[u'Japan'],0,NaN,False,317322,[],800 Two Lap Runners,NaN,0,...,tt0109022,False,NaN,[],1994-07-09,0.000158,800 Two Lap Runners,0,0.0,110.0
2,NaN,[],0,"On November 25th 2005, Heaven 17 put on a rare...",False,100629,[],Heaven 17 - Live at La Scala,NaN,0,...,tt1439310,False,NaN,[],NaN,0.000000,Heaven 17 - Live at La Scala,0,0.0,90.0
3,/hvdR9bJgJsBqZjfaZwI8NdLqIH.jpg,[u'Philippines'],0,"Don Victorino Hernandez, a creole secular prie...",False,256951,"[u'Drama', u'Crime', u'History', u'Horror']",Angustia,NaN,0,...,tt3326152,False,NaN,"[u'Cinema One Originals', u'Cine Rinconada']",2013-11-13,0.002508,Angustia,0,0.0,0.0
4,/aQBgs3f4jc5hJtgMHQcFPoug42s.jpg,[u'France'],0,"""Bartosch showed that animation could be poeti...",False,105773,[u'Animation'],The Idea,NaN,6,...,tt0145834,False,/nRuC7ODVBapTQOanxbtFVLNcGws.jpg,[],1932-01-01,0.000608,L'idée,0,6.4,25.0
5,NaN,[u'Germany'],0,NaN,False,287104,[u'Drama'],Der Fall Maurizius,NaN,0,...,tt0162804,False,NaN,[u'Chamier Film'],1981-09-20,0.000266,Der Fall Maurizius,0,0.0,NaN
6,/5D2Lzw82q4YlvjrWL6vsbJMCmTD.jpg,[u'United States of America'],0,Terry faces off against a much younger Batman,False,269246,"[u'Action', u'Animation', u'Science Fiction']",Batman Beyond Darwyn Cooke's Batman 75th Anniv...,NaN,7,...,tt3643216,False,/ssUR1IOGVP4Mdu7lzcSiyCJNa5b.jpg,"[u'DC Comics', u'Warner Bros. Animation']",2014-04-19,0.120770,Batman Beyond Darwyn Cooke's Batman 75th Anniv...,0,8.3,1.0
7,NaN,[],0,Shows a large open barge loaded with people of...,False,190702,[],Emigrants Landing at Ellis Island,NaN,1,...,tt0419726,False,NaN,[],1903-08-14,0.000143,Emigrants Landing at Ellis Island,0,7.0,2.0
8,NaN,[u'Germany'],0,NaN,False,410551,[u'Drama'],Hütet eure Töchter!,NaN,0,...,tt0058219,False,NaN,[],1964-05-28,0.000151,Hütet eure Töchter!,0,0.0,NaN
9,/pwC2zotRlRuhABg3j8Z7TaYk4Hw.jpg,"[u'France', u'Italy']",0,Stuntman Johnny gets tired of putting his life...,False,3695,"[u'Action', u'Drama']",Stuntman,NaN,0,...,tt0063652,False,NaN,[u'Marianne Productions'],1968-01-01,0.072975,Stuntman,0,0.0,98.0


In [13]:
# import iMDB data
df_imdb = pd.read_csv('imdb_top100_data_parse.txt',index_col=0)
df_imdb

,Unnamed: 0.1,Unnamed: 0.1,title,genres,director,distributors,year,rating,votes,runtimes,...,canonical title,editorial department,canonical title.1,long imdb title,long imdb canonical title,smart canonical title,smart long imdb canonical title,full-size cover url,imdb_ids,mpaa_reason
0,0,0,Four Rooms,['Comedy'],"['0025978', '0734319', '0001675', '0000233']","['0051881', '0014703', '0080682', '0022594', '...",1995.0,6.7,81543.0,"['98', '110::(workprint version)']",...,Four Rooms,"['0020317', '0003692', '0042901', '0120656', '...",Four Rooms,Four Rooms (1995),Four Rooms (1995),Four Rooms,Four Rooms (1995),https://images-na.ssl-images-amazon.com/images...,113101,"pervasive strong language, sexuality and some ..."
1,1,1,"Sonntag, im August",['Short'],['1262754'],NaN,2004.0,6.5,11.0,['15'],...,"Sonntag, im August",NaN,"Sonntag, im August","Sonntag, im August (2004)","Sonntag, im August (2004)","Sonntag, im August","Sonntag, im August (2004)",https://images-na.ssl-images-amazon.com/images...,425473,NaN
2,2,2,Star Wars,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",['0000184'],"['0000756', '0074275', '20', '0105985', '20', ...",1977.0,8.7,969951.0,"['121', '125::(special edition)']",...,Star Wars,"['0091250', '0300879', '0366165', '0385492', '...",Star Wars,Star Wars (1977),Star Wars (1977),Star Wars,Star Wars (1977),https://images-na.ssl-images-amazon.com/images...,76759,sci-fi violence and brief mild language
3,3,3,Finding Nemo,"['Animation', 'Adventure', 'Comedy', 'Family']","['0004056', '0881279']","['0358618', '0064622', '0000779', '0110125', '...",2003.0,8.1,743705.0,['100'],...,Finding Nemo,"['0023224', '1404655', '1733701', '1404694', '...",Finding Nemo,Finding Nemo (2003),Finding Nemo (2003),Finding Nemo,Finding Nemo (2003),https://images-na.ssl-images-amazon.com/images...,266543,NaN
4,4,4,The Dark,"['Horror', 'Mystery', 'Thriller']",['0269502'],"['0002257', '0059064', '0022594', '0049085', '...",2005.0,5.4,9415.0,"['93', 'Philippines:92::(cut)']",...,"Dark, The","['1262315', '3024002', '1930271', '0996933', '...","Dark, The",The Dark (2005),"Dark, The (2005)","Dark, The","Dark, The (2005)",https://images-na.ssl-images-amazon.com/images...,411267,some violent/disturbing images and language
5,5,5,Metropolis,"['Drama', 'Sci-Fi']",['0000485'],"['0034406', '0021661', '0023400', '0103079', '...",1927.0,8.3,119193.0,"['153', 'Germany:210::(premiere cut)', 'German...",...,Metropolis,"['6258563', '8357205']",Metropolis,Metropolis (1927),Metropolis (1927),Metropolis,Metropolis (1927),https://images-na.ssl-images-amazon.com/images...,17136,NaN
6,6,6,My Life Without Me,"['Drama', 'Romance']",['0170043'],"['0059453', '0004490', '0030891', '0115739', '...",2003.0,7.6,21762.0,"['106', 'Argentina:107']",...,My Life Without Me,"['0006971', '0545480', '0647711']",My Life Without Me,My Life Without Me (2003),My Life Without Me (2003),My Life Without Me,My Life Without Me (2003),https://images-na.ssl-images-amazon.com/images...,314412,language
7,7,7,Pirates of the Caribbean: The Curse of the Bla...,"['Action', 'Adventure', 'Fantasy']",['0893659'],"['0044374', '0044279', '0038978', '0148456', '...",2003.0,8.0,853508.0,['143'],...,Pirates of the Caribbean: The Curse of the Bla...,"['0067659', '2064189', '0135440', '2093951', '...",Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...,https://images-na.ssl-images-amazon.com/images...,325980,action/adventure violence
8,8,8,Kill Bill: Vol. 1,"['Action', 'Crime', 'Thriller']",['0000233'],"['0022594', '0054762', '0102291', '0127137', '...",2003.0,8.1,780067.0,"['111', 'Japan:112']",...,Kill Bill: Vol. 1,"['0195549', '0195555', '0218986', '2093951', '...",Kill Bill: Vol. 1,Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 1,Kill Bill: Vol. 1 (2003),https://images-na.ssl-images-amazon.com/image

In [14]:
# drop duplicate or non-informative imdb features
imdb_column_drop = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1', 'runtimes', 'cover url', 'language codes', 'kind', 'country codes', 'full-size cover url', 'canonical title', 'canonical title.1', 'long imdb title', 'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title', ]
df_imdb_dropped = df_imdb.drop(imdb_column_drop, axis=1)
df_imdb_dropped.head()

,title,genres,director,distributors,year,rating,votes,languages,producer,mpaa,...,production companies,cinematographer,plot outline,plot,cast,animation department,original music,editorial department,imdb_ids,mpaa_reason
0,Four Rooms,['Comedy'],"['0025978', '0734319', '0001675', '0000233']","['0051881', '0014703', '0080682', '0022594', '...",1995.0,6.7,81543.0,['English'],"['0004744', '0375402', '0483300', '0734319', '...",R,...,"['0022594', '0019267']","['0006554', '0622897', '0005824', '0782900']",Four interlocking tales that take place in a f...,"[u""This movie features the collaborative direc...","['0205423', '0004866', '0000420', '0000187', '...","['0177478', '0005062', '0476021', '0600724', '...",['0249375'],"['0020317', '0003692', '0042901', '0120656', '...",113101,"pervasive strong language, sexuality and some ..."
1,"Sonntag, im August",['Short'],['1262754'],NaN,2004.0,6.5,11.0,['German'],['1262754'],NaN,...,NaN,['0689837'],A couple on a boat. Their love is burnt out. B...,[u'A couple on a boat. Their love is burnt out...,"['0501854', '0920574']",NaN,['1909419'],NaN,425473,NaN
2,Star Wars,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",['0000184'],"['0000756', '0074275', '20', '0105985', '20', ...",1977.0,8.7,969951.0,['English'],"['0476030', '0000184', '0564768']",PG,...,"['0071326', '0000756']",['0852405'],Luke Skywalker joins forces with a Jedi Knight...,"[u'The Imperial Forces, under orders from crue...","['0000434', '0000148', '0000402', '0001088', '...",NaN,['0002354'],"['0091250', '0300879', '0366165', '0385492', '...",76759,sci-fi violence and brief mild language
3,Finding Nemo,"['Animation', 'Adventure', 'Comedy', 'Family']","['0004056', '0881279']","['0358618', '0064622', '0000779', '0110125', '...",2003.0,8.1,743705.0,['English'],"['1380146', '0005124', '0910237']",NaN,...,"['0017902', '0008970']","['0129269', '0489689', '0165995']",After his son is captured in the Great Barrier...,"[u""A clown fish named Marlin lives in the Grea...","['0000983', '0001122', '1071252', '0000353', '...","['1162887', '0054659', '0065086', '0073559', '...",['0002353'],"['0023224', '1404655', '1733701', '1404694', '...",266543,NaN
4,The Dark,"['Horror', 'Mystery', 'Thriller']",['0269502'],"['0002257', '0059064', '0022594', '0049085', '...",2005.0,5.4,9415.0,"['English', 'Welsh']","['0027271', '0093337', '0160091', '0397077', '...",R,...,"['0002257', '0014411', '0112240', '0104811']",['0003791'],In mourning over the tragic drowning of their ...,"[u'While in Wales visiting her husband James, ...","['0004742', '0000293', '0747687', '1552136', '...",NaN,['0125220'],"['1262315', '3024002', '1930271', '0996933', '...",411267,some violent/disturbing images and language


In [15]:
# drop duplicate or non-informative imdb features
tmdb_column_drop = ['adult', 'poster_path', 'original_title', 'video', 'id', 'homepage', 'backdrop_path', 'production_companies', 'production_countries', 'spoken_languages', 'title']
df_tmdb_dropped = df_tmdb.drop(tmdb_column_drop, axis=1)
df_tmdb_dropped.head()

,revenue,overview,genres,tagline,vote_count,belongs_to_collection,original_language,status,imdb_id,release_date,popularity,budget,vote_average,runtime
0,0,NaN,[u'Comedy'],NaN,0,0,es,Released,tt0191090,1972-01-21,0.000005,0,0.0,NaN
1,0,NaN,[],NaN,0,0,ja,Released,tt0109022,1994-07-09,0.000158,0,0.0,110.0
2,0,"On November 25th 2005, Heaven 17 put on a rare...",[],NaN,0,0,en,Released,tt1439310,NaN,0.000000,0,0.0,90.0
3,0,"Don Victorino Hernandez, a creole secular prie...","[u'Drama', u'Crime', u'History', u'Horror']",NaN,0,0,en,Released,tt3326152,2013-11-13,0.002508,0,0.0,0.0
4,0,"""Bartosch showed that animation could be poeti...",[u'Animation'],NaN,6,0,en,Released,tt0145834,1932-01-01,0.000608,0,6.4,25.0


In [16]:
# rename columns in tmdb dataset
for i in range(len(df_tmdb_dropped.columns.values)):
    df_tmdb_dropped.columns.values[i] = df_tmdb_dropped.columns.values[i] + '_TMDB'
df_tmdb_dropped.head()

,revenue_TMDB,overview_TMDB,genres_TMDB,tagline_TMDB,vote_count_TMDB,belongs_to_collection_TMDB,original_language_TMDB,status_TMDB,imdb_id_TMDB,release_date_TMDB,popularity_TMDB,budget_TMDB,vote_average_TMDB,runtime_TMDB
0,0,NaN,[u'Comedy'],NaN,0,0,es,Released,tt0191090,1972-01-21,0.000005,0,0.0,NaN
1,0,NaN,[],NaN,0,0,ja,Released,tt0109022,1994-07-09,0.000158,0,0.0,110.0
2,0,"On November 25th 2005, Heaven 17 put on a rare...",[],NaN,0,0,en,Released,tt1439310,NaN,0.000000,0,0.0,90.0
3,0,"Don Victorino Hernandez, a creole secular prie...","[u'Drama', u'Crime', u'History', u'Horror']",NaN,0,0,en,Released,tt3326152,2013-11-13,0.002508,0,0.0,0.0
4,0,"""Bartosch showed that animation could be poeti...",[u'Animation'],NaN,6,0,en,Released,tt0145834,1932-01-01,0.000608,0,6.4,25.0


In [17]:
# extract all column names from imdb and tmdb datasets for merged dataset
columns = []
for i in range(len(df_imdb_dropped.columns.values)):
    columns += [(df_imdb_dropped.columns.values[i])]
for i in range(len(df_tmdb_dropped.columns.values)):
    columns += [(df_tmdb_dropped.columns.values[i])]

In [18]:
# create a dataframe to store information from both imdb and tmdb datasets
index = range(len(df_imdb_dropped)) 
df_merged = pd.DataFrame(index = index, columns=columns)
df_merged = df_merged.fillna(0)
df_merged.head()

,title,genres,director,distributors,year,rating,votes,languages,producer,mpaa,...,vote_count_TMDB,belongs_to_collection_TMDB,original_language_TMDB,status_TMDB,imdb_id_TMDB,release_date_TMDB,popularity_TMDB,budget_TMDB,vote_average_TMDB,runtime_TMDB
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
id_not_match_list = []
for i in range(len(df_imdb_dropped)):
    if i not in [5551, 12767, 15224, 24829]:   # these movies have trouble finding TMDB info, should remove later
        if str(df_imdb_dropped.imdb_ids[i]) in df_tmdb_dropped.imdb_id_TMDB[i]:
            for j in range(len(df_imdb_dropped.columns.values)):
                df_merged.loc[i, df_imdb_dropped.columns.values[j]] = df_imdb_dropped.loc[i, df_imdb_dropped.columns.values[j]]
            for j in range(len(df_tmdb_dropped.columns.values)):
                df_merged.loc[i, df_tmdb_dropped.columns.values[j]] = df_tmdb_dropped.loc[i, df_tmdb_dropped.columns.values[j]]
        else:
            id_not_match_list += [i]

In [22]:
print "Number of IDs not match:", len(id_not_match_list)

Number of IDs not match: 0


#### 2.2.2 Merge genres from TMDB and IMDB

In [23]:
# function genre_assign
# input: genres for one movie, string type
# output: a string list, which are corresponding columns names for the genres of input movie 
def genre_assign(genre_temp):
    columns_list = ['Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy', 'Family', 'Romance', 'Horror', 'Reality-TV', 'Western', 'Documentary', 'Biography', 'News', 'Film-Noir', 'Drama', 'Animation', 'Sci-Fi', 'Thriller', 'Short', 'Mystery', 'Sport', 'War', 'History', 'Game-Show', 'Music', 'Musical', 'Talk-Show', 'Foreign', 'TV movie']
    genre_list = []
    for i in columns_list:
        if i in genre_temp:
            genre_list += [i]
    if "Sci-Fi" not in genre_list:
        if "Science Fiction" in genre_temp:
            genre_list += ['Sci-Fi']
    return genre_list

In [24]:
# create a dataframe to store genres
columns = ['Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy', 'Family', 'Romance', 'Horror', 'Reality-TV', 'Western', 'Documentary', 'Biography', 'News', 'Film-Noir', 'Drama', 'Animation', 'Sci-Fi', 'Thriller', 'Short', 'Mystery', 'Sport', 'War', 'History', 'Game-Show', 'Music', 'Musical', 'Talk-Show', 'Foreign', 'TV movie']
index = range(len(df_merged))
df_merge_y = pd.DataFrame(index = index, columns=columns)
df_merge_y = df_merge_y.fillna(0)

In [25]:
for i in range(len(df_merged)):
    genre_temp = str(df_tmdb.loc[i, "genres"]) + str(df_imdb.loc[i, "genres"])
    column_list_temp = genre_assign(genre_temp)
    for column in column_list_temp:
        df_merge_y.loc[i, column] = 1
df_merge_y.head()

,Action,Adventure,Comedy,Crime,Fantasy,Family,Romance,Horror,Reality-TV,Western,...,Mystery,Sport,War,History,Game-Show,Music,Musical,Talk-Show,Foreign,TV movie
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,1,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [26]:
# remove genres from merged dataframe
df_merged = df_merged.drop(["genres", "genres_TMDB"], axis = 1)

In [ ]:
# write the dataframe out
df_merged.to_csv(('feature_multi_25K.csv'))
df_merge_y.to_csv('genre_multi_25K.csv')

---
## Part3. Feature Engineering

For demenstration of code:
use variable `director` for the top 100 data

### 2.1 Variables regarding people/companies involved production (DONE)

#### 2.1.1 extract important top X IDs (for each genre)


#### Directors

In [11]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 directors associated with each genre

def directors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["director"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    director = features2['director']
        
    ##### list of directors simply by expanding out each of the 100 movie entries 
    director_list = []
    for i in range(len(director)):
        if director[i] > 0:
            for j in range(len(director[i])):
                director_list = np.append(director_list, director[i][j]) #list of directors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    director_genre = pd.DataFrame(index = director_list, columns = genres.columns)
    count = 0

    for i in range(len(director)):
        if director[i] > 0:
            for j in range(len(director[i])):
                director_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    director_full = pd.DataFrame(index = np.unique(director_list), columns = genres.columns)

    for i in range(len(np.unique(director_list))):
        director_full.ix[i, ] = director_genre[director_genre.index == np.unique(director_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    director_full = director_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = director_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = director_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_director=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_director.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_director)

#### Distributors

In [12]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 distributors associated with each genre

def distributors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["distributors"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    distributors = features2['distributors']
        
    ##### list of distributors simply by expanding out each of the 100 movie entries 
    distributors_list = []
    for i in range(len(distributors)):
        if distributors[i] > 0:
            for j in range(len(distributors[i])):
                distributors_list = np.append(distributors_list, distributors[i][j]) #list of distributors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    distributors_genre = pd.DataFrame(index = distributors_list, columns = genres.columns)
    count = 0
    for i in range(len(distributors)):
        if distributors[i] > 0:
            for j in range(len(distributors[i])):
                distributors_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    distributors_full = pd.DataFrame(index = np.unique(distributors_list), columns = genres.columns)
    for i in range(len(np.unique(distributors_list))):
        distributors_full.ix[i, ] = distributors_genre[distributors_genre.index == np.unique(distributors_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    distributors_full = distributors_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = distributors_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = distributors_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_distributors=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_distributors.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_distributors)

#### Cast

In [13]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 cast members associated with each genre

def cast_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["cast"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    cast = features2['cast']
        
    ##### list of distributors simply by expanding out each of the 100 movie entries 
    cast_list = []
    for i in range(len(cast)):
        if cast[i] > 0:
            for j in range(len(cast[i])):
                cast_list = np.append(cast_list, cast[i][j]) #list of distributors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    cast_genre = pd.DataFrame(index = cast_list, columns = genres.columns)
    count = 0
    for i in range(len(cast)):
        if cast[i] > 0:
            for j in range(len(cast[i])):
                cast_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    cast_full = pd.DataFrame(index = np.unique(cast_list), columns = genres.columns)
    for i in range(len(np.unique(cast_list))):
        cast_full.ix[i, ] = cast_genre[cast_genre.index == np.unique(cast_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    cast_full = cast_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = cast_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = cast_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_cast=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_cast.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_cast)

In [14]:
directors_top5 = directors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt")
distributors_top5 = distributors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt")
cast_top5 = cast_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt")

### 2.2 Variables for text analysis

by Zecai

In [ ]:
# val_group2 = ["title","plot outline", "plot", "mpaa_reason", "tagline_TMDB"]

In [ ]:
#### 2.2.1 bag-of-words

In [ ]:
#### 2.2.2 dimention reduction

output: 

a data frame from column `title` that contains that first i PCs and each column is names "title_pci"

## Other discusions to include

The exact form of X and Y depends on the ideas you had previously. In general though Y should involve the genre of a movie, and X the features you want to include to predict the genre. Remember from the lecture that more features does not necessarily equal better prediction performance. Use your application knowledge and the insight you gathered from your genre pair analysis and additional EDA to design Y. Do you want to include all genres? Are there genres that you assume to be easier to separate than others? Are there genres that could be grouped together? There is no one right answer here. We are looking for your insight, so be sure to describe your decision process in your notebook.

In preparation for the deep learning part we strongly encourage you to have two sets of training data X, one with the metadata and one with the movie posters. Make sure to have a common key, like the movie ID, to be able to link the two sets together. Also be mindful of the data rate when you obtain the posters. Time your requests and choose which poster resolution you need. In most cases w500 should be sufficient, and probably a lower resolution will be fine.

The notebook to submit this week should at least include:

Discussion about the imbalanced nature of the data and how you want to address it

Description of your data

What does your choice of Y look like?

Which features do you choose for X and why?

How do you sample your data, how many samples, and why?

Important: You do not need to upload the data itself to Canvas.

#### Discussion about the imbalanced nature of the data and how you want to address it

We notice that there is a imbalance in the amount of data available in each genre. For example, there are a lot of Action and Adventure movies, but very few moviews belonging to genres like Documentary, Music, etc..

To deal with this imbalancce of data, we plan to sample a large amount of movies in our training model among the  25,000 movies that we have downloaded from IMDb and TMDb. We can either sample down, by randomly sampling n samples from each genre, where n is the number of films in the genre with minimum number of films; or we can sample up, by randomly sampling m samples from each genre, where m is a large number, for genres that have movie entries fewer than m, we would use bootstrapping to sample from the available data. The value of m to use depends on the actual computation power of our computers, but should be as large as possible. We would start with a resonable number of m, for instance, 10,000, and increase the number if the model performs poorly. We should also note that we plan to combine some of the highly correlated genres into one meta-genre (e.g., Family and Animation), the distribution of the number of movies in each genre will change, and we will take this into consideration in carrying out sampling.

#### Description of your data

Description of your data (danqing)
We randomly sampled 25,000 movie entires from the TMDb database and used the correponding imdb_ids to extract information from the IMDb database for the same movies. We then merged the two database together and formed our own database with 25,000 entires of movies.

#### What does your choice of Y look like?

#### Q. What does your choice of Y look like?

##### 1. combine IMDB and TMDB
For now we take the union of genres from IMDB and TMDB databse. 
From the EDA in Milestone 1 we see that movies have around 1-7 genres, and in total there are 29 kinds of generes (we sampled 25k data points so our data should cover all the possible genres in the database).

So the raw response would be a vector of length 29, with binary coding of 0 or 1.


##### 2. group genres based on correlation matrix
The other form of Y we want to explore is to group certain genres to reduce the length of the response vector. 


From EDA in Milestone 1 we do see some strong correlations among genres. Below is the correlation matrix of 500 movies we sampled in Milestone 1:
< img src="corr plot_popular500.png">
The correlation matrix built from the 25k data would give us a even more accurate correlation.

We can then set a threshold for the correlation between different genres (for example: group `family` and `animation` since their correlation is above 0.8). We'll try several different threshold for grouping.

After grouping, the Y would be a shorter vector (the exact lengths depends on the threshold and needs tuning in cross validation) with binary coding.

#### Which features do you choose for X and why?

Primarily, we get all our data from IMDB and TMDB. We combine all information by API requests from these 2 sources. After combining, we remove duplicated features such as title, language codes, runtimes, country codes. Further, we deleted features which are non-informative, such as kind, cover url, poster_path, id, backdrop_path. IMDB_ID is also non-informative for prediction but we keep it as a common link between poster names and our dataset. Some other duplicated features are also removed, such as language codes (it is the same as language), country codes (same as countries) and so on.

After these, remaining features for prediction are: 'title', 'director', 'distributors', 'year', 'rating', 'votes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'countries', 'aspect_ratio', 'production companies', 'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music', 'editorial department', 'imdb_ids', 'mpaa_reason', 'revenue_TMDB', 'overview_TMDB', 'tagline_TMDB', 'vote_count_TMDB', 'belongs_to_collection_TMDB', 'original_language_TMDB', 'status_TMDB',  'release_date_TMDB', 'popularity_TMDB', 'budget_TMDB', 'vote_average_TMDB' and 'runtime_TMDB'.
 
 We have shown in Milestone 1 that some of above selected features are strongly correlated with genres, such as 'year of release', 'ratings', 'votes' and so on. We haven't test some of the above features yet but we choose them based on our common sense.
 
One thing to note is that here we only excluded features we are sure duplicate or non-informative. We will further check remaining features when we model on our dataset.

#### How do you sample your data, how many samples, and why?

We get our data about movies from both IMDB and TMDB. There are about 2 million and 400,000 movies in IMDB and TMDB, respectively. However, these amount of data points is too many for us, both for storage and for computation. We don't find any pattern of IMDB IDs thus we don't have a clear clue to effectively sample a portion of IMDB IDs. However, for TMDB IDs, it's sequential, so we can randomly pick some numbers (we used 25K here) in the range of 0 to the largest number. After getting 25K random TMDB IDs, we can query each of them and get information about each movie, including corresponding IMDB IDs. Thus we can query IMDB IDs and combine information of these 2 datasets.

We sampled 25K samples because we think this number would be enough for all our prediction methods but not too many to be a computational burden. However, we are open to any necessary changes to this number, if this amount is not enough, we can get more data points. If it takes too much time to training a model, we can randomly delete some of these movies.

If this number becomes a computational burden, we can also try to sample data points from these 25K dataset to test and optimize our models. For this purpose, we can start with sampling maybe 1000, do tests and decide if we should or if we can increase this number.

Another perspective of sampling is to tackle imbalance nature of the genres. We can sample out equal amount of movies in each genre for our training and testing datasets. We think 500 movies should be enough for each genre.
